In [1]:
import sys
import logging, os
from time import time
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
%matplotlib notebook

import ipyparallel as ipp

from bem import Electrodes, Sphere, Cylinder, Mesh, Grid, Configuration, Result
from bem.formats import stl
from bem.bemColors_lib.bemColors import bemColors
from helper_functions import helper_functions
from collections import OrderedDict

## Rename

In [2]:
# base file name for outputs and inputs is the script name
prefix = "Calcium_trap_planar_cood_fixed v2"

a0 = 75e-6    # Distance from ion to electrode is 40 µm.
unit = 100e-6   # unit of the stl file is µm.
# unit /= 10    # normalize, so mesh.plot will give coordinate numbers in mm

In [3]:
# load electrode faces from colored stl
# s_nta is intermediate processed stl file.
s_nta = stl.read_stl(open("%s.stl" % prefix, "rb"))
print("Import stl:",os.path.abspath("./"+prefix+".stl"),"\n")
print("Electrode colors (numbers):\n")
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=a0/unit,rename={1:"1"}))   
# after scale, unit is a0

Import stl: C:\Users\Photonics\Documents\Working\Photonics\Calcium_trap_planar_cood_fixed v2.stl 

Electrode colors (numbers):

dropping 20083
dropping 5345
dropping 10095
dropping 19501
dropping 15925
dropping 7275
dropping 13683
dropping 1647
dropping 13363
dropping 28135
dropping 19943
dropping 11517
dropping 28463
dropping 1399
dropping 10151
dropping 15669
dropping 5493
dropping 11893
dropping 5997
dropping 5941
dropping 27949
dropping 9893
dropping 3565
dropping 15905
dropping 25969
mesh.gather() not working properly, be sure to have valid input


In [4]:
ele_col = bemColors(np.array(list(set(s_nta[2]))),('fusion360','export_stl'))
ele_col.set_my_color(value = (178,178,178),cl_format = ('fusion360','export_stl','RGBA64'),name = 'self_defined')
ele_col.print_stl_colors()

COLORS in the stl:
['bem6']
['bem13']
['bem19']
['bem17']
['bem11']
['bem15']
['bem1']
['bem22']
['bem12']
['bem5']
['bem18']
['bem3']
['bem16']
['bem7']
['bem9']
['bem23']
['bem4']
['bem2']
['bem20']
['bem10']
['bem14']
['bem24']
['bem8']
['bem21']
['_unkColor0']
TOTAL COLORS:  25


In [5]:
# # assign a name for each color
ele_col.color_electrode(color = 'bem1',name = 'cutout1')
ele_col.color_electrode(color = 'bem2',name = 'cutout2')
ele_col.color_electrode(color = 'bem3',name = 'GND')
ele_col.color_electrode(color = 'bem4',name = 'DC1')
ele_col.color_electrode(color = 'bem5',name = 'DC2')
ele_col.color_electrode(color = 'bem6',name = 'DC3')
ele_col.color_electrode(color = 'bem7',name = 'DC4')
ele_col.color_electrode(color = 'bem8',name = 'DC5')
ele_col.color_electrode(color = 'bem9',name = 'DC6')
ele_col.color_electrode(color = 'bem10',name = 'DC7')
ele_col.color_electrode(color = 'bem11',name = 'DC8')
ele_col.color_electrode(color = 'bem12',name = 'DC9')
ele_col.color_electrode(color = 'bem13',name = 'DC10')
ele_col.color_electrode(color = 'bem14',name = 'RF')
ele_col.color_electrode(color = 'bem15',name = 'DC11')
ele_col.color_electrode(color = 'bem16',name = 'DC12')
ele_col.color_electrode(color = 'bem17',name = 'DC13')
ele_col.color_electrode(color = 'bem18',name = 'DC14')
ele_col.color_electrode(color = 'bem19',name = 'DC15')
ele_col.color_electrode(color = 'bem20',name = 'DC16')
ele_col.color_electrode(color = 'bem21',name = 'DC17')
ele_col.color_electrode(color = 'bem22',name = 'DC18')
ele_col.color_electrode(color = 'bem23',name = 'DC19')
ele_col.color_electrode(color = 'bem24',name = 'DC20')

# # print colors still with no name. These meshes will be neglected in the following codes
ele_col.drop_colors()

# # read stl into mesh with electrode names
# # unnamed meshes will not be imported at all
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=a0/unit,
    rename=ele_col.electrode_colors, quiet=False))

dropping color ['_unkColor0']
TOTAL COLORS DROPPED:  1
dropping 20083
1 planes in electrode DC14
normals vectors:
 [[ 1.23908823e-16  1.00000000e+00 -0.00000000e+00]]
1 planes in electrode DC1
normals vectors:
 [[ 1.30922521e-16  1.00000000e+00 -0.00000000e+00]]
1 planes in electrode DC13
normals vectors:
 [[1.23908823e-16 1.00000000e+00 2.75132846e-31]]
1 planes in electrode DC18
normals vectors:
 [[1.23908823e-16 1.00000000e+00 5.50265692e-31]]
1 planes in electrode DC4
normals vectors:
 [[1.30922521e-16 1.00000000e+00 0.00000000e+00]]
1 planes in electrode DC7
normals vectors:
 [[ 1.30922521e-16  1.00000000e+00 -0.00000000e+00]]
41 planes in electrode cutout2
normals vectors:
 [[ 8.75317097e-01 -1.78639196e-16 -4.83549327e-01]
 [ 8.87170017e-01  0.00000000e+00 -4.61442709e-01]
 [ 8.98672283e-01  0.00000000e+00 -4.38620657e-01]
 [ 9.09385741e-01  0.00000000e+00 -4.15953815e-01]
 [ 9.19606209e-01  0.00000000e+00 -3.92841488e-01]
 [ 9.29189205e-01 -1.89633683e-16 -3.69604468e-01]
 [ 9.

In [6]:
mesh.triangulate(opts="q20Q",new = False);

start triangulate DC14
final opts q20Qzr
finish triangulate DC14
start triangulate DC1
final opts q20Qzr
finish triangulate DC1
start triangulate DC13
final opts q20Qzr
finish triangulate DC13
start triangulate DC18
final opts q20Qzr
finish triangulate DC18
start triangulate DC4
final opts q20Qzr
finish triangulate DC4
start triangulate DC7
final opts q20Qzr
finish triangulate DC7
start triangulate cutout2
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20

In [7]:
mesh.plot()

## Remesh

In [8]:
#here is the location that will be the reference point for further meshing
#usually, this is the ion's location, specified in a0 unit
zl = 0
# yl = (10.2+70)/(a0*1e6)
# xl = (3175.5+3260.57)/(a0*1e6)
yl =0
xl = 0
rad = 5
    
   
# 2. Local triangulation
# areas_from_constraints specifies sphere/cylinder with finer mesh inside it.
# "inside", "outside" set different mesh densities.
    
# first refine
mesh.areas_from_constraints(Sphere(center=np.array([xl, yl, zl]), radius=rad, inside=1., outside=10)); # Sphere constraint


In [8]:
mesh.triangulate(opts="q20Q", new=False); # retriangulate quality and quiet with areas


start triangulate DC14
final opts q20Qzr
finish triangulate DC14
start triangulate DC1
final opts q20Qzr
finish triangulate DC1
start triangulate DC13
final opts q20Qzr
finish triangulate DC13
start triangulate DC18
final opts q20Qzr
finish triangulate DC18
start triangulate DC4
final opts q20Qzr
finish triangulate DC4
start triangulate DC7
final opts q20Qzr
finish triangulate DC7
start triangulate cutout2
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20

In [9]:
# Plot triangle meshes.
mesh.plot()

In [10]:
# second refine
mesh.areas_from_constraints(Sphere(center=np.array([xl, yl, zl]), radius=rad, inside=0.1, outside=10)) # Sphere constraint
mesh.triangulate(opts="q20Q", new=False) # retriangulate quality and quiet with areas

NameError: name 'xl' is not defined

In [11]:
mesh.plot()
print(rad)

NameError: name 'rad' is not defined

## BEM Simulation

In [12]:
# Define calculation function.
def run_job(args):
    # job is Configuration instance.
    job, grid, prefix = args
    # refine twice adaptively with increasing number of triangles, min angle 25 deg.
    job.adapt_mesh(triangles=4e2, opts="q25Q")
    job.adapt_mesh(triangles=1e3, opts="q25Q")
    # solve for surface charges
    job.solve_singularities(num_mom=4, num_lev=3)
    # get potentials and fields
    result = job.simulate(grid, field=job.name=="RF", num_lev=2)    # For "RF", field=True computes the field.
    print("RF :", job.name=="RF")
    result.to_vtk(prefix)
    print("finished job %s" % job.name)
    return job.collect_charges()

### Define grid (the simulation region)

In [13]:
# in um
gap_top = -7.929
middle_electrode = -77
distance_ref1 = 3816
distance_ref2 = 4496

#define the physical distance I want to simulate, in um
Lx_phy = 24
Ly_phy = 30
Lz_phy = 50

#define the physical step size I want, in um
sx_phy = 1
sy_phy = 1
sz_phy = 1

#center
center_x = -0.025
center_y = 0.12; 
center_z = 0

# grid to evalute potential and fields atCreate a grid in unit of scaled length l. Only choose the interested region (trap center) to save time.
Lx, Ly, Lz = Lx_phy / (a0*1e6), Ly_phy / (a0*1e6), Lz_phy / (a0*1e6)    # in the unit of scaled length l
sx, sy, sz = sx_phy / (a0*1e6), sy_phy / (a0*1e6), sz_phy / (a0*1e6)
# ni is grid point number, si is step size. Thus to fix size on i direction you need to fix ni*si.
nx, ny, nz = [2*np.ceil(L/2.0/ss).astype('int')+1 for ss, L in zip((sx, sy, sz), (Lx, Ly, Lz))]
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(center_x, center_y, center_z), step=(sx, sy, sz), shape=(nx, ny, nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("Grid origin/l:", grid.get_origin()[0])

Size/l: 0.32 0.4 0.6666666666666666
Step/l: 0.013333333333333334 0.013333333333333334 0.013333333333333334
Shape (grid point numbers): 25 31 51
Grid origin/l: -0.185


### Visualize the grid

In [14]:
pv.global_theme.notebook = False
def plot_grid_mesh_stl(grid, mesh):
    plotter = pv.Plotter()
    
    # Convert the mesh to PolyData and add it to the plotter
    pd = mesh.to_polydata()
    plotter.add_mesh(pd, scalars='electrode_name', show_edges=True)
    
    # Add the grid points
    x, y, z = np.meshgrid(grid.to_xyz()[0], grid.to_xyz()[1], grid.to_xyz()[2])
    grid_points = np.vstack([x.ravel(), y.ravel(), z.ravel()]).T
    plotter.add_points(grid_points, color='red', point_size=5)
    
    plotter.show_bounds()
    plotter.add_camera_orientation_widget()
    plotter.show()

# Call the function to plot everything together
plot_grid_mesh_stl(grid, mesh)

### Parallel computation

In [ ]:
jobs = list(Configuration.select(mesh,"RF"))
# evaluate electric potential for each configurations: one electrode at 1V, the rest 0V.

# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
# asyncresult.wait_interactive()
#print("Computing time: %f s"%(time()-t0))
# c.shutdown(hub=True)
if asyncresult.ready():
    print("Computing time: %f s" % (time() - t0))
else:
    print("Tasks are still running")


# jobs = list(Configuration.select(mesh, "RF"))
# # Evaluate electric potential for each configuration: one electrode at 1V, the rest 0V.

# t0 = time()
# # Sequential computation without parallelization
# for job in jobs:
#     run_job((job, grid, prefix))

# print("Computing time: %f s" % (time() - t0))


In [17]:
jobs = list(Configuration.select(mesh, "DC"))
# Evaluate electric potential for each configuration: one electrode at 1V, the rest 0V.

t0 = time()
# Sequential computation without parallelization
for job in jobs:
    run_job((job, grid, prefix))

print("Computing time: %f s" % (time() - t0))


start triangulate DC14
final opts q25Qzra
finish triangulate DC14
start triangulate DC1
final opts q25Qzra
finish triangulate DC1
start triangulate DC13
final opts q25Qzra
finish triangulate DC13
start triangulate DC18
final opts q25Qzra
finish triangulate DC18
start triangulate DC4
final opts q25Qzra
finish triangulate DC4
start triangulate DC7
final opts q25Qzra
finish triangulate DC7
start triangulate cutout2
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25

C:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


start triangulate DC14
final opts q25Qzra
finish triangulate DC14
start triangulate DC1
final opts q25Qzra
finish triangulate DC1
start triangulate DC13
final opts q25Qzra
finish triangulate DC13
start triangulate DC18
final opts q25Qzra
finish triangulate DC18
start triangulate DC4
final opts q25Qzra
finish triangulate DC4
start triangulate DC7
final opts q25Qzra
finish triangulate DC7
start triangulate cutout2
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25

C:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


start triangulate DC14
final opts q25Qzra
finish triangulate DC14
start triangulate DC1
final opts q25Qzra
finish triangulate DC1
start triangulate DC13
final opts q25Qzra
finish triangulate DC13
start triangulate DC18
final opts q25Qzra
finish triangulate DC18
start triangulate DC4
final opts q25Qzra
finish triangulate DC4
start triangulate DC7
final opts q25Qzra
finish triangulate DC7
start triangulate cutout2
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25

...

C:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


start triangulate DC14
final opts q25Qzra
finish triangulate DC14
start triangulate DC1
final opts q25Qzra
finish triangulate DC1
start triangulate DC13
final opts q25Qzra
finish triangulate DC13
start triangulate DC18
final opts q25Qzra
finish triangulate DC18
start triangulate DC4
final opts q25Qzra
finish triangulate DC4
start triangulate DC7
final opts q25Qzra
finish triangulate DC7
start triangulate cutout2
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25

C:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


start triangulate DC14
final opts q25Qzra
finish triangulate DC14
start triangulate DC1
final opts q25Qzra
finish triangulate DC1
start triangulate DC13
final opts q25Qzra
finish triangulate DC13
start triangulate DC18
final opts q25Qzra
finish triangulate DC18
start triangulate DC4
final opts q25Qzra
finish triangulate DC4
start triangulate DC7
final opts q25Qzra
finish triangulate DC7
start triangulate cutout2
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25Qzra
final opts q25

C:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


In [18]:
Result.view(prefix, 'RF') # don't add anything between '' for mesh view

C:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\utilities\misc.py:115: UserWarning: Encountered issue in callback (most recent call last):
  File "C:\Users\Photonics\Documents\Working\bem\bem\electrostatics.py", line 187, in callback_func_iso
    plotter.add_mesh(iso_slider, name = 'iso', cmap = 'Greys', scalars = scalar_name, opacity = 1, show_scalar_bar = False)
  File "C:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\plotting\plotter.py", line 3438, in add_mesh
    raise ValueError('Empty meshes cannot be plotted. Input mesh has zero points.')
ValueError: Empty meshes cannot be plotted. Input mesh has zero points.
  warnings.warn(formatted_exception)
C:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\utilities\misc.py:115: UserWarning: Encountered issue in callback (most recent call last):
  File "C:\Users\Photonics\Documents\Working\bem\bem\electrostatics.py", line 187, in callback_func_iso
    plotter.add_mesh(iso_slider, name = '

In [ ]:
dc_result = Result.from_vtk(prefix, 'DC13')
# dc_result = Result.from_vtk(prefix, 'DC')


grid = dc_result.grid.to_xyz() # access the (x, y, z) coordinates
potential = dc_result.potential # access the simulated potential data
field = dc_result.field # access the simulated field data
field_square = dc_result.field_square # access the simulated field_square data

In [ ]:
print(potential)